In [8]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import cv2

In [74]:
images = {
    size * 10: np.load(f"data/photo_approx_{size}.npy") for size in [50, 100, 200]
}

In [10]:
canny_images = [
    cv2.Canny(cv2.blur(img, (3, 3)), 10, 70, apertureSize=3) for img in images.values()
]
# px.imshow(np.array(canny_images), animation_frame=0, width=5_00).show()

In [11]:
arucoDict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
arucoParams = cv2.aruco.DetectorParameters()
(corners, ids, rejected) = cv2.aruco.detectMarkers(
    images[0], arucoDict, parameters=arucoParams
)
# px.imshow(cv2.aruco.drawDetectedMarkers(images[0], corners, ids))

In [75]:
# 14.3, 14.4 cm shape
marker_size_mm = 144


def guess_focal(img: np.array, dist_mm: int, size_mm: int = marker_size_mm):
    cs = cv2.aruco.detectMarkers(img, arucoDict, parameters=arucoParams)[0][0][0]
    pixel_size = np.mean(
        [
            np.sqrt(np.square(cs[0] - cs[3]).sum()),
            np.sqrt(np.square(cs[1] - cs[2]).sum()),
        ]
    )
    return pixel_size * dist_mm / size_mm


guess_focal(images[500], dist_mm=500)

1175.4708819919163

In [79]:
focals = []
for dist, im in images.items():
    focals.append(guess_focal(im, dist))
    print(f"{dist: >7} | {focals[-1]: >10.2f}")
avg_focal = np.mean(focals)
print(f"\nAverage: {avg_focal}")

    500 |    1175.47
   1000 |    1121.55
   2000 |    1125.00

Average: 1140.67339014124


In [80]:
test_cam_mat = np.array(
    [
        [avg_focal, 0, 0],
        [0, avg_focal, 0],
        [0, 0, 1],
    ]
)

In [81]:
_object_points = np.array(
    [
        [-marker_size_mm / 2, marker_size_mm / 2, 0],
        [marker_size_mm / 2, marker_size_mm / 2, 0],
        [marker_size_mm / 2, -marker_size_mm / 2, 0],
        [-marker_size_mm / 2, -marker_size_mm / 2, 0],
    ]
)


def estimate_pose(cn, cam_internal=test_cam_mat, obj_points=_object_points):
    ret, r_vec, t_vec = cv2.solvePnP(
        obj_points,
        imagePoints=cn,
        cameraMatrix=cam_internal,
        distCoeffs=np.zeros((4,)),
        flags=cv2.SOLVEPNP_IPPE_SQUARE,
    )
    assert ret, "Failed to solve PnP for pose estimation!"
    r_mat = np.empty((3, 3))
    cv2.Rodrigues(r_vec, dst=r_mat)
    return r_mat, t_vec


estimate_pose(corners[0][0])

(array([[ 9.99608497e-01,  5.82761195e-04,  2.79734391e-02],
        [-2.95465208e-04, -9.99507464e-01,  3.13806141e-02],
        [ 2.79779485e-02, -3.13765937e-02, -9.99115981e-01]]),
 array([[140.61564316],
        [214.42427259],
        [478.14046633]]))